In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import yfinance as yf
from KEYS.keys import newsapi, twitter_bearer
import requests
import linkedin_scraper as ls

In [4]:
import requests
import os
import json

bearer_token = twitter_bearer
search_url = "https://api.twitter.com/2/tweets/search/recent"

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def lookup_tweet(company_name):
    # Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
    # expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
    query_params = {'query': f'{company_name} OR #{company_name}','tweet.fields': 'author_id'}

    json_response = connect_to_endpoint(search_url, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))

In [12]:
companies = {'isin':'', 'ticker': '', 'name':'', 'weight':'', 'tweets':[]}

In [23]:
enel = lookup_tweet('ENEL')

200
{
    "data": [
        {
            "author_id": "3347812847",
            "edit_history_tweet_ids": [
                "1636057349113757697"
            ],
            "id": "1636057349113757697",
            "text": "RT @dicoleparolacce: Edoardo \u00e8 fuori dalla casa da una settimana anzi da sei giorni, ad oggi ha fatto solo un post e praticamente tutti i p\u2026"
        },
        {
            "author_id": "397303940",
            "edit_history_tweet_ids": [
                "1636057193631080472"
            ],
            "id": "1636057193631080472",
            "text": "RT @MAB_Brasil: \u270a\ud83c\udffc\ud83d\udea9 Hoje, 15, o MAB realizou um ato em den\u00fancia aos altos pre\u00e7os da luz em Fortaleza (CE), junto com a CUT e a Central de M\u2026"
        },
        {
            "author_id": "914629047367487490",
            "edit_history_tweet_ids": [
                "1636057177566724098"
            ],
            "id": "1636057177566724098",
            "text": "RT 

In [24]:
enel_2 = lookup_tweet('ENEL S.P.A.')

200
{
    "data": [
        {
            "author_id": "1478083608267137033",
            "edit_history_tweet_ids": [
                "1634902108766969857"
            ],
            "id": "1634902108766969857",
            "text": "https://t.co/rj1qfcbQMV \ud83d\udccc \u03a3\u03c4\u03b9\u03c2 9 \u039c\u03b1\u03c1\u03c4\u03af\u03bf\u03c5 2023, \u03b7 Enel S.p.A. \u03c5\u03c0\u03ad\u03b3\u03c1\u03b1\u03c8\u03b5 \u03c3\u03c5\u03bc\u03c6\u03c9\u03bd\u03af\u03b1 \u03bc\u03b5 \u03c4\u03b7\u03bd \u0394\u0395\u0397 \u03b3\u03b9\u03b1 \u03c4\u03b7\u03bd \u03c0\u03ce\u03bb\u03b7\u03c3\u03b7 \u03cc\u03bb\u03c9\u03bd \u03c4\u03c9\u03bd \u03bc\u03b5\u03c4\u03bf\u03c7\u03ce\u03bd \u03c0\u03bf\u03c5 \u03ba\u03b1\u03c4\u03ad\u03c7\u03b5\u03b9 \u03bf \u03cc\u03bc\u03b9\u03bb\u03bf\u03c2 Enel \u03c3\u03c4\u03b7 \u03a1\u03bf\u03c5\u03bc\u03b1\u03bd\u03af\u03b1. \n#EnelGreenPower #PPC #energy #economics #renewables #renewableenergy"
        },
        {
            "author_id": "545978235",
            "

In [14]:
url = 'https://it.marketscreener.com'
url_ind = url + '/quotazioni/indice/FTSE-MIB-INDEX-121229431/composizione/'
response_ind = requests.get(url_ind)
soup_ind = BeautifulSoup(response_ind.content, 'html.parser')
table = soup_ind.find('table', {'id': 'ALNI1'})
rows = table.find_all('tr')

In [15]:
def get_ticker(link):
    url_tick = url + link
    response_tick = requests.get(url_tick)
    soup_tick = BeautifulSoup(response_tick.content, 'html.parser')
    info = soup_tick.find_all('div',{'class', 'bc_pos'})[0].find_all('span', {'class':'fvTitleInfo'})
    return info[0].text.strip(), info[1].text.strip()

In [18]:
d = {'isin':[], 'ticker': [], 'name':[], 'weight':[]}

for i in range(1, len(rows)):
    row = rows[i]
    td = row.find_all('td')
    link = td[1].find('a')['href'].strip()
    ticker, isin = get_ticker(link)
    
    d['isin'].append(isin)
    d['ticker'].append(ticker)
    d['name'].append(td[1].text.strip())
    d['weight'].append(td[7].text.strip())

In [19]:
import pandas as pd

df = pd.DataFrame(d)
df

,isin,ticker,name,weight
0,IT0003128367,ENEL,ENEL S.P.A.,10.47%
1,IT0000072618,ISP,INTESA SANPAOLO S.P.A.,9.42%
2,IT0003132476,ENI,ENI S.P.A,7.87%
3,IT0005239360,UCG,UNICREDIT S.P.A.,7.51%
4,NL0000226223,STMPA,STMICROELECTRONICS N.V.,7.05%
5,NL00150001Q9,STLAM,STELLANTIS N.V.,7.04%
6,NL0011585146,RACE,FERRARI N.V.,6.94%
7,IT0000062072,G,GENERALI,6.05%
8,NL0010545661,CNHI,CNH INDUSTRIAL N.V.,3.98%
9,IT0004965148,MONC,MONCLER S.P.A.,2.89%


In [22]:
df.iloc[0,:]

isin      IT0003128367
ticker            ENEL
name       ENEL S.P.A.
weight          10.47%
Name: 0, dtype: object

In [8]:
companies['isin'].append('Recordati')
companies['ticker'].append()
companies['name'].append()
companies['weights'].append()
companies['tweets'].append()


AttributeError: 'dict' object has no attribute 'append'

In [ ]:
msft = yf.Ticker("MSFT")

In [ ]:
msft

In [ ]:
msft.actions

In [ ]:
news = msft.news

In [ ]:
ticker = 'Recordati'
url = (f'https://newsapi.org/v2/everything?q={ticker}&from=2023-01-22&sortBy=publishedAt&apiKey={newsapi}')
response = requests.get(url)

In [ ]:
yf_news = "https://finance.yahoo.com/m/33df1a47-6fb3-336f-9f06-90f567fb0427/dow-jones-rises-ahead-of-fed.html"
yf_resp = requests.get(yf_news)
yf_soup = BeautifulSoup(yf_resp.content, 'html.parser')
art_link = yf_soup.find_all('div', {'class': 'caas-readmore caas-readmore-collapse'})[0].find_all('a', href=True)[0]['href']

In [53]:
art_link

'https://www.investors.com/market-trend/stock-market-today/dow-jones-futures-rise-ahead-of-fed-minutes-nvidia-earnings-on-deck/?src=A00220'

In [54]:
art_resp = requests.get(art_link)
art_soup = BeautifulSoup(art_resp.content, 'html.parser')

In [88]:
url="https://www.borsaitaliana.it/borsa/azioni/documenti/societa-quotate/documenti.html?ndg=9&companyName=A2a&dateFrom=&lang=it"

In [89]:
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')

In [91]:
soup

<!--[if !IE]><!--><!--<![endif]-->
<!--[if !IE]><!--><!--<![endif]-->
<!--[if !IE]><!--><!--<![endif]-->
<!DOCTYPE html>

<html class="no-js" lang="it" xmlns="http://www.w3.org/1999/html">
<head>
<meta content="IE=edge,chrome=1" http-equiv="x-ua-compatible"/>
<!--[if gt IE 9]><!-->
<script src="/media-rwd/assets/js/bit.device.js" type="text/javascript"></script>
<?endif]-->
<meta charset="utf-8"> <title>Documenti - Borsa Italiana</title> <meta name="description" content="Bilanci - Documenti informativi - Progetti di bilancio - Prospetti - Regolamenti - Relazioni - Verbali - Presentazioni">
<meta name="format-detection" content="telephone=no" />
<meta name="viewport" content="user-scalable=yes, initial-scale=1, maximum-scale=1.5, minimum-scale=1, width=device-width, height=device-height" />
<link rel="shortcut icon" href="/media-rwd/assets/images/favicon.ico" type="images/x-icon">
<meta name="apple-mobile-web-app-capable" content="yes" />
<meta name="apple-mobile-web-app-status-bar-styl

In [80]:
table